In [1]:
import os
os.chdir("../")
%pwd

'/Users/tapankheni/Data_Science/Data Science Projects/Wafers_Fault_Prediction'

In [2]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class ModelTrainerConfig:
    root_dir: Path
    x_train_data_path: Path
    y_train_data_path: Path
    model_name: str
    params: dict

In [3]:
from WafersFault.constants import CONFIG_YAML_FILE_PATH, PARAMS_YAML_FILE_PATH, SCHEMA_YAML_FILE_PATH
from WafersFault.utils.common import read_yaml, create_directories, save_json

In [4]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_YAML_FILE_PATH,
        params_filepath = PARAMS_YAML_FILE_PATH,
        schema_filepath = SCHEMA_YAML_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    def get_model_trainer_config(self) -> ModelTrainerConfig:
        config = self.config.model_trainer
        params = self.params.RandomForestClassifier

        create_directories([config.root_dir])

        model_trainer_config = ModelTrainerConfig(
            root_dir=config.root_dir,
            x_train_data_path=config.x_train_data_path,
            y_train_data_path=config.y_train_data_path,
            model_name=config.model_name,
            params=params,
        )
    
        return model_trainer_config

In [5]:
import pandas as pd
import numpy as np
import joblib
from sklearn.ensemble import RandomForestClassifier
from WafersFault import logger
from sklearn.metrics import f1_score, precision_score, recall_score, roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score, cross_val_predict

In [6]:
class ModelTrainer:
    def __init__(self, config: ModelTrainerConfig) -> None:
        self.config = config

    def train(self):
        X_train = np.load(self.config.x_train_data_path)
        y_train = np.load(self.config.y_train_data_path)

        logger.info(f"Shape of X_train: {X_train.shape}")
        logger.info(f"Shape of y_train: {y_train.shape}")
        
        random_classifier = RandomForestClassifier(random_state=42)
        # random_scores = cross_val_score(random_classifier, X_train, y_train, cv=10, scoring="roc_auc", verbose=2)
        random_classifier.fit(X_train, y_train)

        joblib.dump(random_classifier, os.path.join(self.config.root_dir, self.config.model_name))

In [7]:
try:
    config = ConfigurationManager()
    model_trainer_config = config.get_model_trainer_config()
    obj = ModelTrainer(config=model_trainer_config)
    obj.train()

except Exception as e:
    raise e

[2024-03-12 13:04:31,361: INFO: common: yaml file: config/config.yaml loaded successfully]
[2024-03-12 13:04:31,362: INFO: common: yaml file: params.yaml loaded successfully]
[2024-03-12 13:04:31,398: INFO: common: yaml file: schema.yaml loaded successfully]
[2024-03-12 13:04:31,403: INFO: common: created directory at: artifacts]
[2024-03-12 13:04:31,403: INFO: common: created directory at: artifacts/model_trainer]
[2024-03-12 13:04:31,405: INFO: 2462628945: Shape of X_train: (150, 928)]
[2024-03-12 13:04:31,405: INFO: 2462628945: Shape of y_train: (150,)]
